In [44]:
# !pip install openpyxl
!pip install gspread

  Obtaining dependency information for gspread from https://files.pythonhosted.org/packages/17/4b/6280ce9b160286b3909f49ea1a2bd979d1a94d94164ad02edc22ede9da68/gspread-5.11.3-py3-none-any.whl.metadata
Using cached gspread-5.11.3-py3-none-any.whl (46 kB)


In [1]:
import pandas as pd

In [8]:
df1 = pd.read_excel('Test-Input-File.xlsx', sheet_name='SPIEL')

In [11]:
df_validation = df1.isnull()
df_validation

,TEST BED 1\nhttps://10.66.9.88:4432/LinkWorkbench/,TEST BED 2\nhttps://10.237.196.74:4432/LinkWorkbench/,PROD MPID,Message Name,Current Spiel,New Spiel,Status,Character Count
0,False,False,True,False,True,False,True,False
1,False,False,True,False,True,False,True,False
2,False,False,True,False,True,False,True,False
3,False,False,True,False,True,False,True,False
4,False,False,True,False,True,False,True,False
5,False,False,True,False,False,False,True,False


In [17]:
print(all(df_validation['PROD MPID']))  # should be true
print(all(df_validation['Status']))     # should be true
print(any(df_validation['New Spiel']))  # should be false
print(any(df_validation['Message Name']))  #should be false

True
True
False
False


In [22]:
df2 = pd.read_excel('Test-Input-File-bad.xlsx', sheet_name='SPIEL')
df_validation2 = df2.isnull()
df_validation2

,TEST BED 1\nhttps://10.66.9.88:4432/LinkWorkbench/,TEST BED 2\nhttps://10.237.196.74:4432/LinkWorkbench/,PROD MPID,Message Name,Current Spiel,New Spiel,Status,Character Count
0,False,False,True,True,True,False,True,False
1,False,False,True,False,True,True,True,False
2,False,False,True,False,True,False,True,False
3,False,False,True,False,True,False,True,False
4,False,False,True,False,True,False,True,False
5,False,False,False,False,False,False,True,False


In [23]:
print(all(df_validation2['PROD MPID']))  # yields false
print(all(df_validation2['Status']))     # yields true
print(any(df_validation2['New Spiel']))  # yields true
print(any(df_validation2['Message Name']))  # yields true

False
True
True
True


---------------------------------------------------------

In [24]:
from __future__ import print_function

import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google.oauth2 import service_account
from pprint import pprint
import pandas as pd

In [25]:
GSHEET_SCOPE = ['https://www.googleapis.com/auth/spreadsheets']
SERVICE_ACCOUNT_FILE = 'reflected-wind-264114-4b2bba4b561a.json'
spread_id = '1PpyCS3IPHqq2CZ_k3Y7dTs-H-Yty3IQjVTuuJraE2g4'  # spreadsheet

In [26]:
gsheet_credentials = service_account.Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE,
                                                                                   scopes=GSHEET_SCOPE)
service = build("sheets", "v4", credentials=gsheet_credentials)

In [27]:
response = service.spreadsheets().values().get(spreadsheetId=spread_id,
                                                        majorDimension="ROWS",
                                                        range='SPIEL').execute()

In [29]:
columns = response['values'][0]
data = response['values'][1:]
df = pd.DataFrame(data, columns=columns)
df

,TEST BED 1\nhttps://10.66.9.88:4432/LinkWorkbench/,TEST BED 2\nhttps://10.237.196.74:4432/LinkWorkbench/,PROD MPID,Message Name,Current Spiel,New Spiel,Status,Character Count
0,71273,35141,,AMAX_UTANG_P1ANS20,,Pwede mo nang magamit ang iyong hiniram na ANS...,,345
1,71293,35161,,AMAX_UTANG_P1FB10,,Pwede mo nang magamit ang iyong hiniram na FB1...,,261
2,71294,35181,,AMAX_UTANG_P1TIKTOK10,,Pwede mo nang magamit ang iyong hiniram na TIK...,,267
3,71295,35201,,AMAX_UTANG_P1YT10,,Pwede mo nang magamit ang iyong hiniram na YT1...,,260
4,71296,35202,,AMAX_UTANG_P1ML10,,Pwede mo nang magamit ang iyong hiniram na ML1...,,267
5,71297,35221,,AMAX_UTANG_ANS20,Pwede mo nang magamit ang iyong hiniram na UTA...,Pwede mo nang magamit ang iyong hiniram na ANS...,,345


In [43]:
df_validation = df.isin(["", None, 0])
df_validation

,TEST BED 1\nhttps://10.66.9.88:4432/LinkWorkbench/,TEST BED 2\nhttps://10.237.196.74:4432/LinkWorkbench/,PROD MPID,Message Name,Current Spiel,New Spiel,Status,Character Count
0,False,False,True,False,True,False,True,False
1,False,False,True,False,True,False,True,False
2,False,False,True,False,True,False,True,False
3,False,False,True,False,True,False,True,False
4,False,False,True,False,True,False,True,False
5,False,False,True,False,False,False,True,False


In [36]:
len(df['PROD MPID'].iloc[0])

0

In [38]:
df[df['*'].__str__.len() == 0]

AttributeError: 'method-wrapper' object has no attribute 'len'

In [46]:
import gspread

In [48]:
sa = gspread.service_account(filename=SERVICE_ACCOUNT_FILE)
sh = sa.open_by_key(spread_id)
wks = sh.worksheet('SPIEL')
# wks.get_all_records()

In [49]:
df = pd.DataFrame(wks.get_all_records())

In [50]:
df

,TEST BED 1\nhttps://10.66.9.88:4432/LinkWorkbench/,TEST BED 2\nhttps://10.237.196.74:4432/LinkWorkbench/,PROD MPID,Message Name,Current Spiel,New Spiel,Status,Character Count
0,71273,35141,,AMAX_UTANG_P1ANS20,,Pwede mo nang magamit ang iyong hiniram na ANS...,,345
1,71293,35161,,AMAX_UTANG_P1FB10,,Pwede mo nang magamit ang iyong hiniram na FB1...,,261
2,71294,35181,,AMAX_UTANG_P1TIKTOK10,,Pwede mo nang magamit ang iyong hiniram na TIK...,,267
3,71295,35201,,AMAX_UTANG_P1YT10,,Pwede mo nang magamit ang iyong hiniram na YT1...,,260
4,71296,35202,,AMAX_UTANG_P1ML10,,Pwede mo nang magamit ang iyong hiniram na ML1...,,267
5,71297,35221,,AMAX_UTANG_ANS20,Pwede mo nang magamit ang iyong hiniram na UTA...,Pwede mo nang magamit ang iyong hiniram na ANS...,,345


In [51]:
df_validation = df.isin(["", None, 0])
df_validation

,TEST BED 1\nhttps://10.66.9.88:4432/LinkWorkbench/,TEST BED 2\nhttps://10.237.196.74:4432/LinkWorkbench/,PROD MPID,Message Name,Current Spiel,New Spiel,Status,Character Count
0,False,False,True,False,True,False,True,False
1,False,False,True,False,True,False,True,False
2,False,False,True,False,True,False,True,False
3,False,False,True,False,True,False,True,False
4,False,False,True,False,True,False,True,False
5,False,False,True,False,False,False,True,False
